In [2]:
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split

from IPython.display import Audio, display
def allDone():
    urL = 'https://www.wavsource.com/snds_2020-10-01_3728627494378403/animals/cat_meow2.wav'
    display(Audio(url=urL, autoplay=True))
allDone()

random.seed(666)

motifs = np.genfromtxt('data_dev/motifs.csv',dtype='U')
motifxFamMatrix = np.genfromtxt('data_dev/motifxFamMatrix.csv',delimiter=',',dtype=int)
fams = np.genfromtxt('data_dev/fams.csv',dtype='U')

print(len(motifs))

7863


In [3]:
X_train, X_test = train_test_split(range(len(motifs)), test_size=0.15, random_state=666)

train_motifs = motifs[X_train]
test_motifs = motifs[X_test]

## Look at CD-HIT results.

In [6]:
from Bio import SeqIO

results = '1588970924.result/1588970924.fas.db2novel.clstr.sorted'

fasta_sequences = SeqIO.parse(open(results),'fasta')
for i,fasta in enumerate(fasta_sequences):
    name, sequence = fasta.id, str(fasta.seq)
#     print(i,sequence.split())
    if i==1000:
        break

In [7]:
m1 = train_motifs[2268]
m2 = test_motifs[128]

print(m1, m2, m1[7], m2[7])

i1 = np.where(motifs==m1)[0][0]
i2 = np.where(motifs==m2)[0][0]

print(i1,i2)

fi1 = np.where(motifxFamMatrix[i1]==1)[0]
fi2 = np.where(motifxFamMatrix[i2]==1)[0]
print(fi1)
print(fi2)

print(motifxFamMatrix[i1])
print(motifxFamMatrix[i2])

print(fams[fi1])
print(fams[fi2])

_MAARRITQETFDAV AVGTRRGSPLLIGVR T S
1089 5346
[7]
[0]
[0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0]
['PIKK']
['PKA']


## Find Hamming-near motifs between test and train sets.

In [8]:
### https://biology.stackexchange.com/questions/23523/hamming-distance-between-two-dna-strings
def inv_hamming_distance_COPYPASTA(s1, s2):
    #Return the Hamming distance between equal-length sequences
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(ch1 == ch2 for ch1, ch2 in zip(s1, s2)) / len(s1)

In [9]:
def inv_hamming_distance(s1, s2):
    #Return the Hamming distance between equal-length sequences
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    zipped = zip(s1,s2)
    ### don't want to count "empty" characters ...
    max_buff_length = max( sum([1 for c in s1 if c=='_']), sum([1 for c in s2 if c=='_'])  )
    return (sum((ch1 == ch2 and ch1 != '_' and ch2 != '_') for ch1, ch2 in zip(s1, s2)) / 
            (len(s2)-max_buff_length) )

In [10]:
t1 = 'GEDEESESD______'
t2 = 'AEEKEAKSD______'

print(inv_hamming_distance(t1,t2))
print(inv_hamming_distance_COPYPASTA(t1,t2))

print(inv_hamming_distance_COPYPASTA('GEDEESESD','AEEKEAKSD'))
print(inv_hamming_distance('GEDEESESD','AEEKEAKSD'))

0.4444444444444444
0.6666666666666666
0.4444444444444444
0.4444444444444444


In [12]:
import time
import itertools

start = time.time()
similar_motifs = []
for i,combo in enumerate(itertools.product(train_motifs, test_motifs)):
    score = inv_hamming_distance(combo[0],combo[1])
    if score >= 0.6:
#         print(score, combo)
        similar_motifs.append( [combo[0], combo[1], score] )
print("%5.3f mins" % ((time.time()-start)/60))
allDone()


0.596 mins


In [13]:
sim_train = list(set([x[0] for x in similar_motifs]))
sim_test = list(set([x[1] for x in similar_motifs]))

print(len(sim_train))
print(len(sim_test))

276
221


In [14]:
sim_train

['NFLRRRLSDSSFIAN',
 'CLRMGNLYDIDEDQM',
 'KSKPKDASQRRRSLE',
 '____MSSSPLSKKRR',
 'GYKASRPSQENFSLI',
 'KKKFRTPSFLKKSKK',
 'VKLQTPNTFPKRKKG',
 'RRQAERMSQIKRLLS',
 'GETLPDSTPLGLYLK',
 'DDIDLFGSDDEEESE',
 'DVYDKVDYLSSLGKT',
 'NSMRRNNSMRRSNSI',
 'LPRPRSCTWPLPRPE',
 'PALEVSDSESDEALV',
 'PIGEDEESESD____',
 'AVATRRGSPLLIGVR',
 'SRFNRRVSVCAETYN',
 'VSGSHQGSIQELSTI',
 'LFRVSEHSSPEEESS',
 'SPKYSPTSPTYSPTT',
 'ETVNLPPSPPPSPAS',
 'YVVKRRDSATSFSLD',
 'VDRRRAATLREKRRL',
 'SDSEDDSSEFDEDDW',
 'VVALTKTYATAEPFI',
 'NRFTRRASVCAEAYN',
 'KEPSEVPTPKRPRGR',
 'ERAELNQSEEPEAGE',
 'GQRDSSYYWEIEASE',
 'EDPAKFKSIKTGRGP',
 'NRVKRRPSPYEMEIT',
 'PAPIRRRSSNYRAYA',
 'FYYEILNSPEKACSL',
 'PRPLRRESEI_____',
 'QAAYYGQTPGPGGPQ',
 'LQYYYSSSEDEDSDH',
 'RHIVRKRTLRRLLQE',
 'SPTYSPTSPVYTPTS',
 'GDDEDACSDTEATEA',
 'MVPGGDRSPSRMLPP',
 'CTERFVCSPDEVMDT',
 'GRIRRRASQLKVKIP',
 'THLAWINTPRKEGGL',
 'YVPVAPPTPAWQPEI',
 'SLLKKRDSFRRDSKL',
 'PSVPPGGSPLLDSSH',
 'RKKDEGSYDLGERKP',
 'IQSNLDFSPVNSASS',
 'APVRRRSSANYRAYA',
 'SRSLYSSSPGGAYVT',


In [15]:
MY_IDX = 16

t1 = similar_motifs[MY_IDX][0]
t2 = similar_motifs[MY_IDX][1]

i1 = np.where(motifs==t1)[0][0]
i2 = np.where(motifs==t2)[0][0]

print(i1,i2)

fi1 = np.where(motifxFamMatrix[i1]==1)[0]
fi2 = np.where(motifxFamMatrix[i2]==1)[0]
print(fi1)
print(fi2)

print(motifxFamMatrix[i1])
print(motifxFamMatrix[i2])

print(fams[fi1])

print(fams[fi2])

2336 3430
[1]
[0]
[0 1 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]
['AKT']
['PKA']


## Define and save training set indices to remove ... 

In [16]:
idc_to_remove = []
for motif in sim_train:
    idx = np.where(motifs==motif)[0][0]
    idc_to_remove.append(idx)
idc_to_remove = np.array((idc_to_remove))

In [17]:
idc_to_keep = [x for x in X_train if x not in idc_to_remove]
X_train = idc_to_keep

train_motifs = motifs[X_train]
test_motifs = motifs[X_test]

train_motifxFamMatrix = motifxFamMatrix[X_train]
test_motifxFamMatrix = motifxFamMatrix[X_test]

data_dir = "data_dev/"

df = pd.DataFrame(train_motifs,dtype='U')
df.to_csv(data_dir + 'train_motifs.csv',header=None,index=None)
df = pd.DataFrame(test_motifs,dtype='U')
df.to_csv(data_dir + 'test_motifs.csv',header=None,index=None)

df = pd.DataFrame(train_motifxFamMatrix,dtype=int)
df.to_csv(data_dir + 'train_motifxFamMatrix.csv',header=None,index=None)
df = pd.DataFrame(test_motifxFamMatrix,dtype=int)
df.to_csv(data_dir + 'test_motifxFamMatrix.csv',header=None,index=None)

### Getting rid of that bad motif.

In [22]:
# train_motifs = np.genfromtxt('data_dev/train_motifs.csv',dtype='U')
# train_motifxFamMatrix = np.genfromtxt('data_dev/train_motifxFamMatrix.csv',delimiter=',',dtype=int)
# test_motifs = np.genfromtxt('data_dev/test_motifs.csv',dtype='U')
# test_motifxFamMatrix = np.genfromtxt('data_dev/test_motifxFamMatrix.csv',delimiter=',',dtype=int)

# fams = np.genfromtxt('data_dev/fams.csv',dtype='U')

# all_motifs = np.hstack([train_motifs,test_motifs])
# all_motifxFamMatrix = np.vstack([train_motifxFamMatrix,test_motifxFamMatrix])

In [23]:
# np.where(test_motifs=='__PKRKVSSAEGAAK')

In [24]:
# poo = [x for x in range(len(test_motifs)) if x!=????]
# new_test_motifs = test_motifs[poo]
# new_test_motifxFamMatrix = test_motifxFamMatrix[poo]

# print(len(test_motifs))
# print(len(new_test_motifs))

# data_dir = "data_dev/"

# df = pd.DataFrame(new_test_motifs,dtype='U')
# df.to_csv(data_dir + 'new_test_motifs.csv',header=None,index=None)
# df = pd.DataFrame(new_test_motifxFamMatrix,dtype='U')
# df.to_csv(data_dir + 'new_test_motifxFamMatrix.csv',header=None,index=None)

## Meta-data

In [27]:
print("train motifs:",len(train_motifs))
print("test motifs:",len(test_motifs))
print("total motifs:",len(test_motifs)+len(train_motifs))

train motifs: 6407
test motifs: 1180
total motifs: 7587
